## Demostración del servicio (Speech to text) Amazon Polly


Aquí verás un resumen de las características que posee Polly.
El texto de entrada del servicio puede ser puro o en formato SSML. El formato SSML es un estándar W3C. Este formato sirve para describir la forma en que un texto debe ser leído por el sintetizador. Con los siguientes ejemplos quedará más claro cómo utilizar SSML.


In [ ]:
# checking the boto3 version
import boto3
if boto3.__version__ < "1.4.8":
    !conda update --yes boto3
    print("Please, reload your kernel now.")

In [ ]:
import boto3
import numpy as np
import base64
import os
import time

from io import BytesIO
from IPython.core.display import display, HTML, Javascript, display_javascript

Después de importar las bibliotecas, vamos a crear un cliente para Polly y un mixer. El mixer se utilizará para la reproducción del audio retornado por Polly.

In [ ]:
client = boto3.client('polly')

In [ ]:
html = """
<script src="https://ajax.googleapis.com/ajax/libs/jquery/3.2.1/jquery.min.js"></script>
<audio id="audio"><source id="media" src="audio.mp3" type="audio/mpeg" /></audio>
<script type="text/javascript">    
    function play_audio(){
        $("#audio")[0].load();
        $("#audio")[0].play();
    } // play_audio
</script>
"""
HTML(html)

Ahora vamos a definir algunas funciones auxiliares para:
- tocar un streaming de audio
- sintetizar el habla a partir de un archivo SSML
- cargar un archivo del disco

In [ ]:
def play_audio(audiostream):
    audio = open('audio.mp3', 'wb+')
    audio.write(audiostream.read())
    audio.flush()
    audio.close()
    
    jso = Javascript('play_audio()')
    display_javascript(jso)

In [ ]:
def sintetiza(ssml, lexicons=[], voiceid='Enrique' ):
    if os.path.isfile('audio.mp3'):
        os.remove('audio.mp3') 
    response = client.synthesize_speech(
        LexiconNames=lexicons,
        OutputFormat='mp3',
        SampleRate='22050',
        Text=ssml,
        TextType='ssml',
        VoiceId=voiceid
    )
    play_audio( response['AudioStream'] )
    print(ssml)

In [ ]:
def load_file( filename ):
    return "".join( open( filename, 'r').readlines() )

Ok. Ahora vamos a hacer una breve prueba de lectura. Tenga en cuenta las tags &lt;speak&gt; e &lt;p&gt;. Ellas forman parte del formato SSML.

In [ ]:
sintetiza( load_file( 'poema.es.ssml' ) )

### Some SSML tags, supported by Polly
   - &lt;break&gt;: Intervalo en seg o mili
   - &lt;lang&gt;: Corrige la pronunciación de un extranjerismo
   - &lt;mark&gt;: Agrega marcas en el texto sin cambiar la reproducción
   - &lt;p&gt;: Configura un nuevo párrafo con una pausa.
   - &lt;phoneme&gt;: Interpreta fonemas
   - &lt;prosody&gt;: Regula la cadencia del habla: velocidad, entonación, etc
   - &lt;s&gt;: Da una pausa después de leer la oración
   - &lt;say-as&gt;: Cambia la forma de leer un texto: hora, teléfono, etc
   - &lt;sub&gt;: Sustituye un término por una sentencia
   - &lt;w&gt;: Cambia la pronunciación de una palabra
   - &lt;amazon:effect name = "whispered"&gt;: Sussurrar
   - &lt;amazon:effect name = "drc"&gt;: Resaltar sonido

More info: 
http://docs.aws.amazon.com/polly/latest/dg/supported-ssml.html


### P & Lang

In [ ]:
sintetiza( load_file( 'lang.es.ssml' ) )

## Break

In [ ]:
sintetiza( load_file( 'break.es.ssml' ) )

## Say as

In [ ]:
sintetiza( load_file( 'sayas.es.ssml' ) )

## Lexicons

#### Carga el archivo de de lexicons

In [ ]:
acronimos = load_file( 'lexicons.es.pls' )
print(acronimos)
response = client.put_lexicon(
    Name='acronimos',
    Content=acronimos
)

In [ ]:
sotaques = {
    'uai': 'uai.es.pls',
    'tche': 'tche.es.pls',
    'meu': 'meu.es.pls'
}
for nome, nome_arquivo in sotaques.items():
    response = client.put_lexicon(
        Name=nome,
        Content=load_file( nome_arquivo )
    )

In [ ]:
sintetiza( load_file( 'lexicon.es.ssml' ) )

In [ ]:
sintetiza( load_file( 'lexicon.es.ssml' ), [ 'acronimos'] )

In [ ]:
sintetiza( load_file( 'agua.es.ssml' ), [ 'acronimos'] )

#### Acento Mineirês

In [ ]:
sintetiza( load_file( 'lexicon_sotaques.es.ssml' ), [ 'uai'] )

#### Acento Paulistês

In [ ]:
sintetiza( load_file( 'lexicon_sotaques.es.ssml' ), [ 'meu'] )

#### Acento Gauchês

In [ ]:
sintetiza( load_file( 'lexicon_sotaques.es.ssml' ), [ 'tche'] )

### Sussurrar y gritar

In [ ]:
sintetiza( load_file( 'alteracao_voz.es.ssml' ) )

### Respirar

In [ ]:
ssml = """
<speak>
     Con grandes poderes<amazon:breath duration="short" volume="x-loud"/>
     vienen grandes responsabilidades
</speak>
"""
sintetiza(ssml)

In [ ]:
ssml = """
<speak>
   <amazon:auto-breaths>
Un Anillo para gobernarlos a todos. Un Anillo para encontrarlos,
un Anillo para atraerlos a todos y atarlos en las tinieblas.

En los días posteriores a la batalla de los Campos del Pelennor, 
Gandalf aconsejó lanzar un ataque directo a Mordor, para vaciarlo 
de las tropas allí acampadas y desviar la atención de Sauron de la llegada de Frodo y Sam.
   </amazon:auto-breaths>
</speak>
"""
sintetiza(ssml)

# Hack the following code and use the latest feature (Timbre)
http://docs.aws.amazon.com/polly/latest/dg/supported-ssml.html#vocaltractlength-tag


In [ ]:
# English voices: F(Joanna Salli Kimberly Kendra Ivy Matthew) M(Justin Joey)
# Spanish voices: F(Penelope) M(Miguel)
# Portuguese voices: F(Vitoria) M(Ricardo)
ssml = """
<speak>
 your text  <amazon:effect vocal-tract-length="+15%"> with the effect effect effect </amazon:effect> here
</speak>
"""
sintetiza(ssml, voiceid='Ivy')